This iPyNotebook will run the dilutions and the IPTG induction of cells for the tRNA depletion experiment.

In [1]:
import opentrons.simulate
import os
from rich import print
import random

protocol_file = 'complete_protocol.py'
pre_data = 'cr_sample_pre.csv'
iptg_volume = 6 # In microliters. 6ul of 0.00625M IPTG is 0.01875mM IPTG in 200ul total

First, we'll define the locations of each cell on the plate. 

In [2]:
cell_locations = {
    'B2': ['MCH10', 'RBS0.125'],
    'C2': ['MCH25', 'RBS0.125'],
    'D2': ['MCH50', 'RBS0.125'],
    'E2': ['MCH75', 'RBS0.125'],

    'B3': ['MCH10', 'RBS0.25'],
    'C3': ['MCH25', 'RBS0.25'],
    'D3': ['MCH50', 'RBS0.25'],
    'E3': ['MCH75', 'RBS0.25'],

    'B4': ['MCH10', 'RBS0.5'],
    'C4': ['MCH25', 'RBS0.5'],
    'D4': ['MCH50', 'RBS0.5'],
    'E4': ['MCH75', 'RBS0.5'],

    'B5': ['MCH10', 'RBS1'],
    'C5': ['MCH25', 'RBS1'],
    'D5': ['MCH50', 'RBS1'],
    'E5': ['MCH75', 'RBS1'],

    'B6': ['MCH10', 'RBS2'],
    'C6': ['MCH25', 'RBS2'],
    'D6': ['MCH50', 'RBS2'],
    'E6': ['MCH75', 'RBS2'],

    'B7': ['MCH10', 'RBS4'],
    'C7': ['MCH25', 'RBS4'],
    'D7': ['MCH50', 'RBS4'],
    'E7': ['MCH75', 'RBS4'],

    'B8': ['blank', 'blank'],
    'C8': ['blank', 'blank'],
    'D8': ['blank', 'blank'],
    'E8': ['blank', 'blank'],
}

Now, we'll assign each preconditioning well to two experimental wells.

In [3]:
import random
random.seed('day 1')

valid_rows = list("BCDEFG")
valid_columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

valid_wells = [f"{row}{column}" for row in valid_rows for column in valid_columns]

wells_with_cells = [well for well in cell_locations if cell_locations[well] != ['blank', 'blank']]

final_positions = {}

for well in wells_with_cells:
    position_1 = random.choice(valid_wells)
    position_1_index = valid_wells.index(position_1)
    del(valid_wells[position_1_index])
    position_2 = random.choice(valid_wells)
    position_2_index = valid_wells.index(position_2)
    del(valid_wells[position_2_index])

    final_positions[position_1] = [well, cell_locations[well][0], cell_locations[well][1]]
    final_positions[position_2] = [well, cell_locations[well][0], cell_locations[well][1]]

# Fill the rest of the valid positions with blanks

for well in valid_wells:
    final_positions[well] = final_positions.get(well, ['blank', 'blank', 'blank'])


Next, we'll calculate the appropriate well dilutions for the experiment using the data from the plate reader.

In [9]:
import pandas as pd

def parse_platereader(filename):
    """Parse the output of a Teccan plate reader and make the data tidy"""
    with open(filename, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    dataframe = pd.DataFrame(columns=['time', 'well', 'label', 'value', 'temperature'])
    datasets = []
    reading_dataset = [None, None]
    start_time = None
    for i, line in enumerate(lines):
        line = line.split(',')
        if line[0].startswith('End Time:'):
            continue
        elif line[0].startswith('Start Time:'):
            start_time = i
        elif start_time and any(line) > 0 and not any(reading_dataset):
            reading_dataset[0] = i
        elif start_time and any(line) == 0 and any(reading_dataset):
            reading_dataset[1] = i # Exclusive
            datasets.append(reading_dataset)
            reading_dataset = [None, None]

    def filter_function(test_value):
        if test_value == "":
            return False
        else: 
            return True

    for dataset in datasets:
        data = lines[dataset[0]:dataset[1]]
        label = data.pop(0).split(',')[0]

        if data[0].startswith('Cycle'):
            cycle = data.pop(0).split(',')[1:]

        times = list(filter(filter_function, data.pop(0).split(',')[1:]))
        temperature = list(filter(filter_function, data.pop(0).split(',')[1:]))
        data = list(filter(filter_function, [line.split(',') for line in data]))

        for line in data:
            well = str(line.pop(0))
            line = list(filter(filter_function, line))
            for i, value in enumerate(line):
                adding_frame = pd.DataFrame({
                    'time': float(times[i]),
                    'well': well,
                    'label': label,
                    'value': float(value),
                    'temperature': float(temperature[i]),
                    }, index=[0])


                dataframe = pd.concat([dataframe, adding_frame])

    return dataframe

platereader_file = pre_data
platereader_data = parse_platereader(platereader_file)


target_od600 = 0.0234506  # This is equivilant to 0.2 OD600 on our 1cm spectrophotometer

blank_wells = [x for x in cell_locations.keys() if 'blank' in cell_locations[x]]

cell_volumes = {}
lb_volumes = {}


# Get the latest timepoint from the platereader data
latest_timepoint = platereader_data['time'].max()

# Set the OD600 column to be a float


# Get the latest OD600 values for the blank wells
blank_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'].isin(blank_wells)) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()

# Get all the wells that are used in the protocol
occupied_wells = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['label'].isin(['OD600'])), 'well']
# Remove the blank wells
occupied_wells = [well for well in occupied_wells if well not in blank_wells]
# Only include wells we have a key for
occupied_wells = [well for well in occupied_wells if well in cell_locations.keys()]

for start_well in occupied_wells:
    well_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'] == start_well) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()
    well_od600 = well_od600 - blank_od600

    cell_volume = (target_od600 / well_od600) * 200
    if cell_volume > (200-iptg_volume):
        cell_volume = (200-iptg_volume)
    if cell_volume < 0:
        cell_volume = 0
    lb_volume = (200-iptg_volume) - cell_volume

    cell_volumes[start_well] = [start_well, cell_volume]
    lb_volumes[start_well] = lb_volume


/tmp/ipykernel_25754/3625994426.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, adding_frame])


[
    'B2',
    'B3',
    'B4',
    'B5',
    'B6',
    'B7',
    'C2',
    'C3',
    'C4',
    'C5',
    'C6',
    'C7',
    'D2',
    'D3',
    'D4',
    'D5',
    'D6',
    'D7',
    'E2',
    'E3',
    'E4',
    'E5',
    'E6',
    'E7'
]

Next, we associate the calculated cell volumes and LB volumes with the final well positions

In [5]:
for well in final_positions:
    source_well = final_positions[well][0]

    if source_well == 'blank':
        final_positions[well].append(0)
        final_positions[well].append((200-iptg_volume))
    else:
        final_positions[well].append(cell_volumes[source_well][1])
        final_positions[well].append(lb_volumes[source_well])

print(final_positions)

{
    'D7': ['B2', 'MCH10', 'RBS0.125', 46.901199999999996, 147.0988],
    'B9': ['B2', 'MCH10', 'RBS0.125', 46.901199999999996, 147.0988],
    'C9': ['C2', 'MCH25', 'RBS0.125', 46.901199999999996, 147.0988],
    'E3': ['C2', 'MCH25', 'RBS0.125', 46.901199999999996, 147.0988],
    'D6': ['D2', 'MCH50', 'RBS0.125', 46.901199999999996, 147.0988],
    'C2': ['D2', 'MCH50', 'RBS0.125', 46.901199999999996, 147.0988],
    'F6': ['E2', 'MCH75', 'RBS0.125', 46.901199999999996, 147.0988],
    'B4': ['E2', 'MCH75', 'RBS0.125', 46.901199999999996, 147.0988],
    'G7': ['B3', 'MCH10', 'RBS0.25', 46.901199999999996, 147.0988],
    'F11': ['B3', 'MCH10', 'RBS0.25', 46.901199999999996, 147.0988],
    'B7': ['C3', 'MCH25', 'RBS0.25', 46.901199999999996, 147.0988],
    'E7': ['C3', 'MCH25', 'RBS0.25', 46.901199999999996, 147.0988],
    'E6': ['D3', 'MCH50', 'RBS0.25', 46.901199999999996, 147.0988],
    'G11': ['D3', 'MCH50', 'RBS0.25', 46.901199999999996, 147.0988],
    'D9': ['E3', 'MCH75', 'RBS0.25', 46.901199999999996, 147.0988],
    'E10': ['E3', 'MCH75', 'RBS0.25', 46.901199999999996, 147.0988],
    'E2': ['B4', 'MCH10', 'RBS0.5', 46.901199999999996, 147.0988],
    'C11': ['B4', 'MCH10', 'RBS0.5', 46.901199999999996, 147.0988],
    'F2': ['C4', 'MCH25', 'RBS0.5', 46.901199999999996, 147.0988],
    'C3': ['C4', 'MCH25', 'RBS0.5', 46.901199999999996, 147.0988],
    'F5': ['D4', 'MCH50', 'RBS0.5', 46.901199999999996, 147.0988],
    'G5': ['D4', 'MCH50', 'RBS0.5', 46.901199999999996, 147.0988],
    'D2': ['E4', 'MCH75', 'RBS0.5', 46.901199999999996, 147.0988],
    'G10': ['E4', 'MCH75', 'RBS0.5', 46.901199999999996, 147.0988],
    'D10': ['B5', 'MCH10', 'RBS1', 46.901199999999996, 147.0988],
    'B3': ['B5', 'MCH10', 'RBS1', 46.901199999999996, 147.0988],
    'F3': ['C5', 'MCH25', 'RBS1', 46.901199999999996, 147.0988],
    'B10': ['C5', 'MCH25', 'RBS1', 46.901199999999996, 147.0988],
    'E8': ['D5', 'MCH50', 'RBS1', 46.901199999999996, 147.0988],
    'D4': ['D5', 'MCH50', 'RBS1', 46.901199999999996, 147.0988],
    'G3': ['E5', 'MCH75', 'RBS1', 46.901199999999996, 147.0988],
    'G9': ['E5', 'MCH75', 'RBS1', 46.901199999999996, 147.0988],
    'B8': ['B6', 'MCH10', 'RBS2', 46.901199999999996, 147.0988],
    'F4': ['B6', 'MCH10', 'RBS2', 46.901199999999996, 147.0988],
    'B6': ['C6', 'MCH25', 'RBS2', 46.901199999999996, 147.0988],
    'F7': ['C6', 'MCH25', 'RBS2', 46.901199999999996, 147.0988],
    'E11': ['D6', 'MCH50', 'RBS2', 46.901199999999996, 147.0988],
    'C4': ['D6', 'MCH50', 'RBS2', 46.901199999999996, 147.0988],
    'D11': ['E6', 'MCH75', 'RBS2', 46.901199999999996, 147.0988],
    'C6': ['E6', 'MCH75', 'RBS2', 46.901199999999996, 147.0988],
    'C10': ['B7', 'MCH10', 'RBS4', 46.901199999999996, 147.0988],
    'G4': ['B7', 'MCH10', 'RBS4', 46.901199999999996, 147.0988],
    'D8': ['C7', 'MCH25', 'RBS4', 46.901199999999996, 147.0988],
    'C5': ['C7', 'MCH25', 'RBS4', 46.901199999999996, 147.0988],
    'E5': ['D7', 'MCH50', 'RBS4', 46.901199999999996, 147.0988],
    'D5': ['D7', 'MCH50', 'RBS4', 46.901199999999996, 147.0988],
    'C8': ['E7', 'MCH75', 'RBS4', 46.901199999999996, 147.0988],
    'G6': ['E7', 'MCH75', 'RBS4', 46.901199999999996, 147.0988],
    'B2': ['blank', 'blank', 'blank', 0, 194],
    'B5': ['blank', 'blank', 'blank', 0, 194],
    'B11': ['blank', 'blank', 'blank', 0, 194],
    'C7': ['blank', 'blank', 'blank', 0, 194],
    'D3': ['blank', 'blank', 'blank', 0, 194],
    'E4': ['blank', 'blank', 'blank', 0, 194],
    'E9': ['blank', 'blank', 'blank', 0, 194],
    'F8': ['blank', 'blank', 'blank', 0, 194],
    'F9': ['blank', 'blank', 'blank', 0, 194],
    'F10': ['blank', 'blank', 'blank', 0, 194],
    'G2': ['blank', 'blank', 'blank', 0, 194],
    'G8': ['blank', 'blank', 'blank', 0, 194]
}

Now we add the dictionaries to the base protocol and test to make sure it runs.

In [6]:
# Make a copy of base_protocol.py and name it complete_protocol.py
os.system('cp base_protocol_multichannel.py complete_protocol.py')

# Add the volumes to the complete_protocol.py file
with open('complete_protocol.py', 'a') as file:
    file.write(f'final_positions = {final_positions}\n')
    file.write(f'iptg_volume = {iptg_volume}\n')

# Simulate the protocol
opentrons.simulate.simulate(open(protocol_file))

/home/croots/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/home/croots/.opentrons/deck_calibration.json not found. Loading defaults


([{'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'location': H1 of Opentrons 96 Tip Rack 300 µL on 8,
    'text': 'Picking up tip from H1 of Opentrons 96 Tip Rack 300 µL on 8'},
   'logs': [<LogRecord: opentrons.calibration_storage.ot2.tip_length, 30, /home/croots/miniconda3/envs/opentrons/lib/python3.10/site-packages/opentrons/calibration_storage/ot2/tip_length.py, 53, "Tip length calibrations not found for None">]},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 51.591319999999996,
    'location': Location(point=Point(x=23.28, y=155.68, z=4.5), labware=B2 of NEST 96 Well Plate 200 µL Flat on 4),
    'rate': 1.0,
    'text': 'Aspirating 51.591319999999996 uL from B2 of NEST 96 Well Plate 200 µL Flat on 4 at 150.0 uL/sec'},
   'logs': []},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 46.901199999999996,
    'location': Location(point=Point(x=